# Centre wise Monthly SQ to SR conversion

In [29]:
import pyodbc
import pandas as pd
import psycopg2
import time 
from datetime import datetime
import calendar
import openpyxl

In [30]:
sql_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=SDPL-IC-PBGW-VM;'
    'DATABASE=SdplWarehouse;'
    'UID=lisadmin;'
    'PWD=abcd-1234'
)

In [15]:
cdf = pd.read_sql("""WITH T1 AS (SELECT INVENTSITEID, FORMAT(INVOICEDATE, 'MMM-yy') AS FM,
                     SUM(CASE WHEN TESTCODE = 'POL00033' OR TESTCODE = 'POL00011' THEN AMOUNT END) AS Consultfees
                     FROM [VW_BI_ITEMSALES_FY_2025-26]
                     GROUP BY INVENTSITEID, FORMAT(INVOICEDATE, 'MMM-yy')),
	                 T2 AS (SELECT INVENTSITEID, FORMAT(APPOINTMENTDATE, 'MMM-yy') AS FM,
                     COUNT(DISTINCT CASE WHEN BOOKINGSTATUS = 2 AND SALESIDREF != '' THEN PATIENTID END) AS TotalOpdPatients
                     FROM SDPLOPDAPPOINTMENTSLOT_MOD
                     WHERE APPOINTMENTDATE >= '2025-04-01'
                     GROUP BY INVENTSITEID, FORMAT(APPOINTMENTDATE, 'MMM-yy'))

SELECT T1.INVENTSITEID, T1.FM, T1.Consultfees, T2.TotalOpdPatients
FROM T1 
LEFT JOIN T2
ON T1.INVENTSITEID = T2.INVENTSITEID AND T1.FM = T2.FM""", sql_conn)

cdf = cdf.sort_values(by= 'INVENTSITEID', ascending= True)
cdf.head(7)

C:\Users\suradmin\AppData\Local\Temp\ipykernel_3544\3726055018.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cdf = pd.read_sql("""WITH T1 AS (SELECT INVENTSITEID, FORMAT(INVOICEDATE, 'MMM-yy') AS FM,


,INVENTSITEID,FM,Consultfees,TotalOpdPatients
14,ASN,May-25,80700.0,83.0
79,ASN,Apr-25,94561.0,98.0
138,ASN,Jun-25,149280.0,190.0
105,BAN,Jun-25,8600.0,39.0
33,BAN,May-25,4200.0,44.0
159,BAN,Apr-25,6300.0,31.0
165,BAR,Apr-25,209540.0,380.0


In [16]:
sdf = pd.read_sql("""SELECT INVENTSITEID, FORMAT(APPOINTMENTDATE, 'MMM-yy') AS FM,
                     COUNT(DISTINCT CASE WHEN ISONLYMEDICINE = 0 THEN PATIENTID END) AS TotalPrescriptionsWithTests,
                     SUM(TOTALBILLEDAMOUNT) AS TotalPrescribedAmount,
                     SUM(TOTALSRAMOUNT) AS TotalSRAmount,
                     CASE WHEN SUM(TOTALBILLEDAMOUNT) = 0 THEN 0 ELSE SUM(TOTALSRAMOUNT) * 100.0 / SUM(TOTALBILLEDAMOUNT)  END AS SRConvPerc
                     FROM SDPLOPDAPPFOLLOWUP
                     WHERE APPOINTMENTDATE >= '2025-04-01'
                     GROUP BY INVENTSITEID, FORMAT(APPOINTMENTDATE, 'MMM-yy')""", sql_conn)

sdf = sdf.sort_values(by= 'INVENTSITEID', ascending= True)
adf = pd.merge(cdf, sdf, left_on= ['INVENTSITEID', 'FM'], right_on= ['INVENTSITEID', 'FM'], how= 'left')
adf = adf[['INVENTSITEID', 'FM', 'TotalOpdPatients', 'Consultfees', 'TotalPrescriptionsWithTests', 
           'TotalPrescribedAmount', 'TotalSRAmount', 'SRConvPerc']]
adf.head(5)

C:\Users\suradmin\AppData\Local\Temp\ipykernel_3544\3376702977.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sdf = pd.read_sql("""SELECT INVENTSITEID, FORMAT(APPOINTMENTDATE, 'MMM-yy') AS FM,


,INVENTSITEID,FM,TotalOpdPatients,Consultfees,TotalPrescriptionsWithTests,TotalPrescribedAmount,TotalSRAmount,SRConvPerc
0,ASN,May-25,83.0,80700.0,66.0,180958.0,57020.00,31.510074
1,ASN,Apr-25,98.0,94561.0,80.0,233440.0,106089.94,45.446341
2,ASN,Jun-25,190.0,149280.0,156.0,540310.0,293200.00,54.265144
3,BAN,Jun-25,39.0,8600.0,29.0,109940.0,54860.00,49.899945
4,BAN,May-25,44.0,4200.0,17.0,96840.0,54975.00,56.768897


In [18]:
def opd_score(row):
    if row['SRConvPerc'] > 40:
        return 5
    elif row['SRConvPerc'] > 35:
        return 4
    elif row['SRConvPerc'] > 30:
        return 3
    else:
        return 0

adf['opd_score'] = adf.apply(opd_score, axis= 1)
adf.head(10)

,INVENTSITEID,FM,TotalOpdPatients,Consultfees,TotalPrescriptionsWithTests,TotalPrescribedAmount,TotalSRAmount,SRConvPerc,opd_score
0,ASN,May-25,83.0,80700.0,66.0,180958.0,57020.00,31.510074,3
1,ASN,Apr-25,98.0,94561.0,80.0,233440.0,106089.94,45.446341,5
2,ASN,Jun-25,190.0,149280.0,156.0,540310.0,293200.00,54.265144,5
3,BAN,Jun-25,39.0,8600.0,29.0,109940.0,54860.00,49.899945,5
4,BAN,May-25,44.0,4200.0,17.0,96840.0,54975.00,56.768897,5
5,BAN,Apr-25,31.0,6300.0,28.0,103115.0,27160.00,26.339523,0
6,BAR,Apr-25,380.0,209540.0,213.0,909300.0,387923.00,42.661717,5
7,BAR,Jun-25,271.0,201200.0,171.0,666297.0,414982.00,62.281835,5
8,BAR,May-25,349.0,255180.0,244.0,1114888.0,612527.00,54.940675,5
9,BAS,May-25,65.0,24650.0,56.0,128130.0,34030.00,26.558963,0


# Doctor wise SQ to SR conversion

In [33]:
ddf = pd.read_sql("""WITH T1 AS (SELECT FORMAT(INVOICEDATE, 'MMM-yy') AS FM, ACX_REFERREDDOCTOR,
                     SUM(CASE WHEN TESTCODE = 'POL00033' OR TESTCODE = 'POL00011' THEN AMOUNT END) AS Consultfees
                     FROM [VW_BI_ITEMSALES_FY_2025-26]
                     GROUP BY FORMAT(INVOICEDATE, 'MMM-yy'), ACX_REFERREDDOCTOR),
	                 T2 AS (SELECT FORMAT(APPOINTMENTDATE, 'MMM-yy') AS FM, DOCID,
                     COUNT(DISTINCT CASE WHEN BOOKINGSTATUS = 2 AND SALESIDREF != '' THEN PATIENTID END) AS TotalOpdPatients
                     FROM SDPLOPDAPPOINTMENTSLOT_MOD
                     WHERE APPOINTMENTDATE >= '2025-04-01'
                     GROUP BY FORMAT(APPOINTMENTDATE, 'MMM-yy'), DOCID)

SELECT  T1.FM, T1.ACX_REFERREDDOCTOR, T3.DOCTORNAME, T3.PRACTICETYPEID, T1.Consultfees, T2.TotalOpdPatients
FROM T1 
LEFT JOIN T2
ON  T1.FM = T2.FM AND T1.ACX_REFERREDDOCTOR = T2.DOCID
LEFT JOIN ACX_DOCTORTABLE T3
ON T1.ACX_REFERREDDOCTOR = T3.DOCID
WHERE T1.ACX_REFERREDDOCTOR IS NOT NULL""", sql_conn)

ddf.head(7)

C:\Users\suradmin\AppData\Local\Temp\ipykernel_3544\1869505855.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ddf = pd.read_sql("""WITH T1 AS (SELECT FORMAT(INVOICEDATE, 'MMM-yy') AS FM, ACX_REFERREDDOCTOR,


,FM,ACX_REFERREDDOCTOR,DOCTORNAME,PRACTICETYPEID,Consultfees,TotalOpdPatients
0,May-25,DOC-32007,ANIMESH CHANDAN,,NaN,NaN
1,Apr-25,DOC-00621,TAPOBRATA CHOWDHURY,GENERAL PHYSICIAN,NaN,NaN
2,Jun-25,DOC-05445,S SARKAR,,NaN,NaN
3,May-25,DOC-85797,BUDDHA DEB CHATTERJEE,ORTHOPAEDIC,NaN,NaN
4,May-25,DOC-45739,SHYAM KRISHNAN,,NaN,NaN
5,Jun-25,DOC-54845,MOUMITA GHOSH,,NaN,NaN
6,Apr-25,DOC-02242,SARBARI GHOSH MONDAL,GYNAECOLOGIST,NaN,NaN


In [34]:
ddf.to_excel(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\opd.xlsx", index= False)

In [37]:
fdf = pd.read_sql("""SELECT DOCID, FORMAT(APPOINTMENTDATE, 'MMM-yy') AS FM,
                     COUNT(DISTINCT CASE WHEN ISONLYMEDICINE = 0 THEN PATIENTID END) AS TotalPrescriptionsWithTests,
                     SUM(TOTALBILLEDAMOUNT) AS TotalPrescribedAmount,
                     SUM(TOTALSRAMOUNT) AS TotalSRAmount,
                     CASE WHEN SUM(TOTALBILLEDAMOUNT) = 0 THEN 0 ELSE SUM(TOTALSRAMOUNT) * 100.0 / SUM(TOTALBILLEDAMOUNT)  END AS SRConvPerc
                     FROM SDPLOPDAPPFOLLOWUP 
                     WHERE APPOINTMENTDATE >= '2025-04-01'
                     GROUP BY DOCID, FORMAT(APPOINTMENTDATE, 'MMM-yy')""", sql_conn)

edf = pd.merge(ddf, fdf, left_on= 'ACX_REFERREDDOCTOR', right_on= 'DOCID', how= 'left')
edf = edf[['FM_x', 'ACX_REFERREDDOCTOR', 'DOCTORNAME', 'PRACTICETYPEID', 'TotalOpdPatients', 'Consultfees',
           'TotalPrescriptionsWithTests', 'TotalPrescribedAmount', 'TotalSRAmount', 'SRConvPerc']]
edf.head(10)

C:\Users\suradmin\AppData\Local\Temp\ipykernel_3544\3112387819.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fdf = pd.read_sql("""SELECT DOCID, FORMAT(APPOINTMENTDATE, 'MMM-yy') AS FM,


,FM_x,ACX_REFERREDDOCTOR,DOCTORNAME,PRACTICETYPEID,TotalOpdPatients,Consultfees,TotalPrescriptionsWithTests,TotalPrescribedAmount,TotalSRAmount,SRConvPerc
0,May-25,DOC-32007,ANIMESH CHANDAN,,NaN,NaN,NaN,NaN,NaN,NaN
1,Apr-25,DOC-00621,TAPOBRATA CHOWDHURY,GENERAL PHYSICIAN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jun-25,DOC-05445,S SARKAR,,NaN,NaN,NaN,NaN,NaN,NaN
3,May-25,DOC-85797,BUDDHA DEB CHATTERJEE,ORTHOPAEDIC,NaN,NaN,NaN,NaN,NaN,NaN
4,May-25,DOC-45739,SHYAM KRISHNAN,,NaN,NaN,NaN,NaN,NaN,NaN
5,Jun-25,DOC-54845,MOUMITA GHOSH,,NaN,NaN,NaN,NaN,NaN,NaN
6,Apr-25,DOC-02242,SARBARI GHOSH MONDAL,GYNAECOLOGIST,NaN,NaN,NaN,NaN,NaN,NaN
7,Apr-25,DOC-117797,KHURSHID AHMED,,NaN,NaN,NaN,NaN,NaN,NaN
8,May-25,DOC-02144,SADHAB SAHA,GENERAL PHYSICIAN,NaN,NaN,NaN,NaN,NaN,NaN
9,May-25,DOC-44947,Dr Self Advice,,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
edf.to_excel(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\opd2.xlsx", index= False)